<a href="https://colab.research.google.com/github/ffviana/NUTRECON/blob/main/Simulation/NUTRECON_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NUTRECON General Task Description
## Same-type trials
In same-type trials, subjects must choose between a small but guaranteed quantity of a reward (Reference option) vs a probability (13, 22, 38, 50 or 75%) of winning a fixed amount of the same reward or getting nothing (Lottery Option).

* Lottery probabilities: 13, 22, 38, 50 or 75%
* Quantities:
  * Money:
    * Reference option: 2 €
    * Lottery option: 2, 4.5, 10, 22.5 or 50 €

  * Yogurts A and B	
    * Reference option: 1
    * Lottery option: 1, 2, 5, 11 or 25 cups (containers?)

There are (nº of lottery probs * nº of lottery quantities) unique lottery (UL) options in per reward type. 

We can play around with the number of UL option repetitions. (6 in the paper)

_______
## Mixed-type trials:
In mixed-type trials, subjects must choose between a small but guaranteed amount of Money (Reference option) vs a probability (13, 22, 38, 50 or 75%) of winning a fixed amount of Yogurt A, Yogurt B, Water or getting nothing (Lottery Option).
Quantities:
Money
Reference: 0.20€

* Lottery probabilities: 13, 22, 38, 50 or 75%
* Quantities:
  * Reference option: 0.20 €
  * Lottery option: 1, 2, 3, 4 or 6 cups (of yogurt A or yoburt B)

  * Yogurts A and B	
    * Reference option: 1
    * Lottery option: 1, 2, 5, 11 or 25 cups (containers?)


## Framework

From Levy and Glimcher 2011, the estimated utility (EU) is given by:

\begin{align}
        EU(p_{i},X_{i},α_{i}) = p_{i} × X_{i}^{α_{i}}
\end{align}

where $p_{i}$ is the probability of recieving a quantity $X_{i}$ of reward $i$.  
<br>
From Levy and Glimcher 2011, with $EU_{L}$ and $EU_{R}$ as the EU for lottery and reference options, respcetively, the probability $P_{L}$ of choosing the lottery option is given by:

\begin{align}
        P_{L}(β_{i}) = \frac{1}{1 + e^{β_{i} * (EU_{L} - EU_{R})}}
\end{align}

however, if $EU_{L} > EU_{R} \rightarrow P_{L} < 0.5$. Thus, the correct equation is:
\begin{align}
        P_{L}(β_{i}) = 1 - \frac{1}{1 + e^{β_{i} * (EU_{L} - EU_{R})}}
\end{align}


In [ ]:
# Import Packages
import numpy as np
import pandas as pd
from random import randint
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

## Same-type trials

#### Define Task parameters

In [ ]:
# Same-type task variables

uniqueLott_Nreps= 6                                 # Unique Lottery Repititions  
st_refPs = [1]
st_lottPs = [0.13, 0.22, 0.38, .50, .75]            # Same-type Trials Lottery probabilities

money_id = 'Money'
money_refQs = [2]                                   # Euros
money_lottQs = [2, 4.5, 10, 22.5, 50]               # Euros
money_refPars = [money_id, money_refQs, st_refPs]
money_lottPars = [money_id, money_lottQs, st_lottPs]

yogiCplus_id = 'C+'
yogiCplus_refQs = [1]                               # Cups of C+ yogurt 
yogiCplus_lottQs = [1, 2, 5, 11, 25]                # Cups of C+ yogurt
yogiCplus_refPars = [yogiCplus_id, yogiCplus_refQs, st_refPs]
yogiCplus_lottPars = [yogiCplus_id, yogiCplus_lottQs, st_lottPs]

yogiCminus_id = 'C-'
yogiCminus_refQs = yogiCplus_refQs                  # Cups of C- yogurt 
yogiCminus_lottQs = yogiCplus_lottQs                # Cups of C- yogurt 
yogiCminus_refPars = [yogiCminus_id, yogiCminus_refQs, st_refPs]
yogiCminus_lottPars = [yogiCminus_id, yogiCminus_lottQs, st_lottPs]


### Simulate Data

#### One Subject

##### Initialization parameters

In [ ]:
# %% Subject specific parameters
money_alpha = 0.7
yogiCplus_alpha = 0.8
yogiCminus_alpha = 0.8

money_beta = 1.8
yogiCplus_beta = money_Beta
yogiCminus_beta = money_Beta

alphas = {'Money' : money_alpha,
             'C+' : yogiCplus_alpha,
             'C-' : yogiCminus_alpha}
betas = {'Money' : money_beta,
             'C+' : yogiCplus_beta,
             'C-' : yogiCminus_beta}

##### Simulate Choice Data

###### Get Trial Combinations + alphas and betas

In [ ]:
column_names = ['ref_type', 'ref_qt', 'ref_prob' ,
                'lott_type', 'lottery_qt', 'lottery_prob',
                'ref_alpha', 'lott_alpha', 'beta',
                'ref_EU', 'lott_EU', 'pL', 'choice']

def get_uniqueCombinations(ref_pars, lott_pars):
  ref_uniqueCombs = [ [ref_pars[0],qt,prob] for qt in ref_pars[1] for prob in ref_pars[2] ]
  lott_uniqueCombs = [ [lott_pars[0],qt,prob] for qt in lott_pars[1] for prob in lott_pars[2] ]

  uniqueCombs = [ref + lott for ref in ref_uniqueCombs for lott in lott_uniqueCombs]
  return uniqueCombs

# Generate Unique Lotteries  
money_uniqueLottCombs = get_uniqueCombinations(money_refPars, money_lottPars)
yogiCplus_uniqueLottCombs = get_uniqueCombinations(yogiCplus_refPars, yogiCplus_lottPars)
yogiCminus_uniqueLottCombs = get_uniqueCombinations(yogiCminus_refPars, yogiCminus_lottPars)

allUniqueLotteryCombinations = money_uniqueLottCombs + yogiCplus_uniqueLottCombs + yogiCminus_uniqueLottCombs

LotteryCombinations = allUniqueLotteryCombinations * uniqueLott_Nreps         # Lottery combinations for all reward types

N_trials = len(LotteryCombinations)                 # Total number of Same-type trials

print('Total number of Same-type trials = {}'.format(N_trials))

sameType_sim_df = pd.DataFrame(LotteryCombinations, columns = column_names[:6])

# Get alpha and beta values
sameType_sim_df[column_names[6]] = sameType_sim_df[column_names[0]].replace(alphas)
sameType_sim_df[column_names[7]] = sameType_sim_df[column_names[3]].replace(alphas)
sameType_sim_df[column_names[8]] = sameType_sim_df[column_names[3]].replace(betas)
sameType_sim_df.sample(5)

Total number of Same-type trials = 450


,ref_type,ref_qt,ref_prob,lott_type,lottery_qt,lottery_prob,ref_alpha,lott_alpha,beta
375,Money,2,1,Money,2.0,0.13,0.7,0.7,1.8
442,C-,1,1,C-,11.0,0.38,0.8,0.8,1.8
235,Money,2,1,Money,10.0,0.13,0.7,0.7,1.8
238,Money,2,1,Money,10.0,0.50,0.7,0.7,1.8
285,C-,1,1,C-,5.0,0.13,0.8,0.8,1.8


###### Compute reference and Lottery EUs and calculate pL

In [ ]:
def get_EU_(p,X, alpha):
  return p * X**alpha

def get_EU(row, cols):
  p = row[cols[1]]
  X = row[cols[0]]
  alpha = row[cols[2]]
  EU = get_EU_(p,X, alpha)
  return EU

def get_pL_(euL, euR, beta):
  return 1 - 1/(1 + np.exp(beta * (euL - euR)))

def get_pL(row, cols = column_names[8:11]):
  euL = row[cols[2]]
  euR = row[cols[1]]
  beta = row[cols[0]]
  pL = get_pL_(euL, euR, beta)
  return pL

ref_cols = column_names[1:3] + [column_names[6]]
lott_cols = column_names[4:6] + [column_names[7]]

sameType_sim_df[column_names[9]] = sameType_sim_df.apply(lambda row: get_EU(row, ref_cols), axis=1)
sameType_sim_df[column_names[10]] = sameType_sim_df.apply(lambda row: get_EU(row, lott_cols), axis=1)
sameType_sim_df[column_names[11]] = sameType_sim_df.apply(lambda row: get_pL(row), axis=1)
sameType_sim_df.sample(5)

,ref_type,ref_qt,ref_prob,lott_type,lottery_qt,lottery_prob,ref_alpha,lott_alpha,beta,ref_EU,lott_EU
260,C+,1,1,C+,5.0,0.13,0.8,0.8,1.8,1.000000,0.471107
394,Money,2,1,Money,22.5,0.75,0.7,0.7,1.8,1.624505,6.631153
314,Money,2,1,Money,10.0,0.75,0.7,0.7,1.8,1.624505,3.758904
49,C+,1,1,C+,25.0,0.75,0.8,0.8,1.8,1.000000,9.849479
76,Money,2,1,Money,2.0,0.22,0.7,0.7,1.8,1.624505,0.357391


###### Simulate Choices

In [ ]:
seed = 1
rng = np.random.default_rng(1)

sameType_sim_df[column_names[12]] = sameType_sim_df.apply(lambda row: get_choice(row), axis=1)
sameType_sim_df.sample(5)

,ref_type,ref_qt,ref_prob,lott_type,lottery_qt,lottery_prob,ref_alpha,lott_alpha,beta,ref_EU,lott_EU,pL,choice
108,C+,1,1,C+,2.0,0.50,0.8,0.8,1.8,1.000000,0.870551,0.442010,1
345,C+,1,1,C+,25.0,0.13,0.8,0.8,1.8,1.000000,1.707243,0.781262,1
176,C+,1,1,C+,1.0,0.22,0.8,0.8,1.8,1.000000,0.220000,0.197182,0
232,Money,2,1,Money,4.5,0.38,0.7,0.7,1.8,1.624505,1.089011,0.276100,0
343,C+,1,1,C+,11.0,0.50,0.8,0.8,1.8,1.000000,3.404742,0.986985,1


##### Estimate Parameters

In [ ]:
optimize_cols = column_names[:6]  + [column_names[12]]
def get_likelihood(row, params, cols = optimize_cols):
  ref_type = row[cols[0]]
  lott_type = row[cols[3]]
  if len(params) == 4:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta) = params
    alphas = {'Money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
  elif len(params) == 6:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta_money, beta_Cplus, beta_Cminus) = params
    alphas = {'Money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    betas = {'Money' : beta_money,
             'C+' : beta_Cplus,
             'C-' : beta_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
    beta = betas[lott_type]
  
  choice = row[cols[6]]

  ref_X = row[cols[1]]
  ref_p = row[cols[2]]
  ref_EU = get_EU_(ref_p, ref_X, ref_alpha)
  
  lott_X = row[cols[4]]
  lott_p = row[cols[5]]
  lott_EU = get_EU_(lott_p, lott_X, lott_alpha)

  pL = get_pL_(lott_EU, ref_EU, beta)
  if choice == 1:
    likelihood = pL
  else:
    likelihood = 1 - pL
  return likelihood

def get_negLogLikelihood(params, args):

  (df) = args
  task_cols = optimize_cols
  # compute likelihood of each choice
  likelihood = df.apply(lambda row: get_likelihood(row, params, cols = task_cols), axis=1).values
  # Take negative of logLikelihood for convention
  negloglikelihood = - np.sum(np.log(likelihood))
  return negloglikelihood


###### 3 alphas + 1 beta

In [ ]:
alphaMoney0 = 1
alphaCplus0 = 1
alphaCminus0 = 1
beta0 = 2

args = (df)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, beta0)
res = minimize(get_negLogLikelihood, x0, args=args )
print('{}\n  - parameters: {}'.format(res.message, res.x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log


Optimization terminated successfully.
  - parameters: [0.71529899 0.8626828  0.84339465 1.85326669]


###### 3 alphas + 3 betas

In [ ]:
alphaMoney0 = 1
alphaCplus0 = 1
alphaCminus0 = 1
betaMoney0 = 2
betaCplus0 = 2
betaCminus0 = 2

args = (df)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, betaMoney0, betaCplus0, betaCminus0)
res = minimize(get_negLogLikelihood, x0, args=args )
print('{}\n  - parameters: {}'.format(res.message, res.x))

Optimization terminated successfully.
  - parameters: [0.75035941 0.8230851  0.82070782 1.40217176 2.40480505 2.1533486 ]


#### Group

##### Initialization parameters

In [ ]:
N = 2
Subject_noiseLevel = 0.2
Trial_noiseLevel = 0.2

# %% Subject specific parameters
money_alpha = 0.3
yogiCplus_alpha = 0.3
yogiCminus_alpha = 0.3